In [16]:
import numpy as np
import json, os
from nltk.corpus import wordnet as wn
from math import log
import tqdm
import json
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/adaptsystemlab2019/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/adaptsystemlab2019/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/adaptsystemlab2019/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [17]:
def _getTextFile(langual):
    file_list = glob.glob(f'../data/stopwords/stopwords/*_{langual}.txt')
    files = ",".join(file_list)
    return files

def countMinMaxAver(lines):
    min_len = 10000
    aver_len = 0
    max_len = 0
    for temp in lines:
        aver_len = aver_len + len(temp)
        if len(temp) < min_len:
            min_len = len(temp)
        if len(temp) > max_len:
            max_len = len(temp)
    aver_len = 1.0 * aver_len / len(lines)
    print('min_len : ' + str(min_len))
    print('max_len : ' + str(max_len))
    print('average_len : ' + str(aver_len))

In [18]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('V'):
        return wn.VERB
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    else:
        return None
    
def lemmatization(sentence):
    tokens = word_tokenize(sentence)
    tagged_sent = pos_tag(tokens)
    lemmas_sent = []
    for tag in tagged_sent:
        wordnet_pos = get_wordnet_pos(tag[1]) or wn.NOUN
        lemmas_sent.append((tag[0], wordnet_pos))
    return lemmas_sent

In [4]:
a = 'beautiful adjective possessing beauty'
lemmatization(a)

[('beautiful', 'a'), ('adjective', 'a'), ('possessing', 'n'), ('beauty', 'n')]

In [7]:
sememe_network_dict_en_wiki_5000 = np.load("./data/wiki_skb.npy", allow_pickle=True).tolist()

In [8]:
word_set = set()
for k,_ in sememe_network_dict_en_wiki_5000.items():
    word=k.split(" (")[0]
    word_set.add(word)

In [9]:
len(word_set)

5556928

In [10]:
sememe_network_dict_en_wiki_5000_keys = [i for i in sememe_network_dict_en_wiki_5000.keys()]

In [11]:
len(sememe_network_dict_en_wiki_5000)

6114239

In [14]:
sememe_network_dict_en_wordnet_tagme_5000 = np.load("./data/wordnet_skb.npy", allow_pickle=True).tolist()
sememe_network_cdv_en_wordnet_tagme_5000 = np.load("./data/wordnet_cdv.npy", allow_pickle=True).tolist()

In [15]:
len(sememe_network_dict_en_wordnet_tagme_5000)

168274

In [26]:
title_content_lemmatization_sence = np.load("../data/title_content_lemmatization_sence.npy", allow_pickle=True).tolist()

In [27]:
title_content_lemmatization_sence_keys = [k for k in title_content_lemmatization_sence.keys()]

In [28]:
len(title_content_lemmatization_sence_keys)

960853

In [31]:
title_wiki_373749 = set(title_content_lemmatization_sence_keys) - set(sememe_network_dict_en_wiki_5000_keys)

In [32]:
len(title_wiki_373749)

373749

In [33]:
wiki_dict_373749 = {k:title_content_lemmatization_sence[k] for k in title_wiki_373749}

In [35]:
# get network all definition words sememe_network_dict_en_wordnet_tagme_5000
sememe_network_wordnet = {}
for k,v in sememe_network_dict_en_wordnet_tagme_5000.items():
    index = k.split(".")[-1]
    pos = k.split(".")[-2]
    word = "".join(k.split(".")[:-2])
    if word not in sememe_network_wordnet.keys():
        sememe_network_wordnet[word] = []
    sememe_network_wordnet[word].append((pos,v))

In [36]:
len(sememe_network_wordnet)

125497

In [38]:
title_sememe_raw = {}
for k, v in tqdm.tqdm(wiki_dict_373749.items()):
    sememe = []
    for w in v:
        if w in sememe_network_wordnet.keys():
            sememe.append(w)
    if len(sememe) == 0:
        continue
    title_sememe_raw[k] = sememe
countMinMaxAver(title_sememe_raw.values())

100%|███████████████████████████████| 373749/373749 [00:01<00:00, 283183.35it/s]

min_len : 1
max_len : 389
average_len : 13.091166633603754


In [39]:
len(title_sememe_raw)

287331

In [40]:
sememe_freq = {}
sememe_set = set()
for doc_sememes in title_sememe_raw.values():
    for sememe in doc_sememes:
        sememe_set.add(sememe)
        if sememe in sememe_freq:
            sememe_freq[sememe] += 1
        else:
            sememe_freq[sememe] = 1

sememe_lexion = list(sememe_set)
sememe_lexion_size = len(sememe_lexion)

sememe_id_map = {}
for i in range(sememe_lexion_size):
    sememe_id_map[sememe_lexion[i]] = i

print("sememe lexion size: ", sememe_lexion_size)

sememe lexion size:  41272


In [41]:
doc_word_freq = {}
for doc_id,(_, doc_words) in enumerate(title_sememe_raw.items()):
    for word in doc_words:
        word_id = sememe_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1
            
word_doc_list = {}
for i,(_, doc_words) in enumerate(title_sememe_raw.items()):
    appeared = set()
    for word in doc_words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)
        
word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)
    
tfidf_word2doc_all = {}
for i,(title, doc_words) in enumerate(title_sememe_raw.items()):
    doc_word_set = set()
    for word in doc_words:
        if word in doc_word_set or word == title:
            continue
        j = sememe_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        idf = log(1.0 * len(title_sememe_raw) / word_doc_freq[sememe_lexion[j]])
        tfidf_word2doc_all[key] = freq * idf
        doc_word_set.add(word)

In [42]:
title_list = [word for word in title_sememe_raw.keys()]
tfidf_filter = {}
for title, tfidf in tfidf_word2doc_all.items():
    if tfidf > 4:
        w, d = title.split(",")
        if title_list[int(w)] not in tfidf_filter.keys():
            tfidf_filter[title_list[int(w)]] = []
        tfidf_filter[title_list[int(w)]].append(sememe_lexion[int(d)])
print("all lexion size:", len(tfidf_filter))

all lexion size: 237961


In [43]:
title_id_map = {w:i for i,w in enumerate(title_list)}
max_value = 10
sememe_network_wiki = {}
for word, sememes in tfidf_filter.items():
    if len(sememes) != 0:
        if len(sememes) > max_value:
            sememe_freq_max_value = {}
            for sememe in sememes:
                tfidf_word2doc_all_key = str(title_id_map[word]) +","+ str(sememe_id_map[sememe])
                sememe_freq_max_value[sememe] = tfidf_word2doc_all[tfidf_word2doc_all_key]
            sememe_lower = sorted(sememe_freq_max_value.items(),key=lambda item:item[1],reverse=True)[:max_value]
            sememe_lower = [sememe_set[0] for sememe_set in sememe_lower]
            sememe_network_wiki[word] = sememe_lower
        else:
            sememe_network_wiki[word] = sememes
    else:
        continue

In [44]:
sememe_network_wiki = {k:v for k, v in sememe_network_wiki.items() if len(v) != 0}

In [45]:
print("all lexion size:", len(sememe_network_wiki))

all lexion size: 237961


In [46]:
sum([len(v) for v in sememe_network_wiki.values()])/len(sememe_network_wiki)

4.589163770533827

In [47]:
reconstruct_def = {}
for title, sentence in tqdm.tqdm(sememe_network_wiki.items()):
    sentence = lemmatization(" ".join(sentence))
    for (word, pos) in sentence:
        if word.lower() not in sememe_network_wordnet.keys():
            continue
        if title not in reconstruct_def.keys():
            reconstruct_def[title] = set()
        if pos not in [p for (p, _) in sememe_network_wordnet[word.lower()]]:
            reconstruct_def[title] = reconstruct_def[title] | \
            set([s for (p, s) in sememe_network_wordnet[word.lower()]][0])
        else:
            #print(word)
            #print(type(reconstruct_def[title]))
            #print(type([s for (p, s) in sememe_network_wordnet[word.lower()] if p == pos][0]))
            reconstruct_def[title] = reconstruct_def[title] | \
            set([s for (p, s) in sememe_network_wordnet[word.lower()] if p == pos][0])

100%|█████████████████████████████████| 237961/237961 [01:12<00:00, 3296.89it/s]


In [49]:
np.save("reconstruct_def_long", reconstruct_def)

In [50]:
reconstruct_def_raw = np.load("reconstruct_def_long.npy", allow_pickle=True).tolist()

In [51]:
sememe_freq = {}
sememe_set = set()
for doc_sememes in reconstruct_def_raw.values():
    for sememe in doc_sememes:
        sememe_set.add(sememe)
        if sememe in sememe_freq:
            sememe_freq[sememe] += 1
        else:
            sememe_freq[sememe] = 1

sememe_lexion = list(sememe_set)
sememe_lexion_size = len(sememe_lexion)

sememe_id_map = {}
for i in range(sememe_lexion_size):
    sememe_id_map[sememe_lexion[i]] = i

print("sememe lexion size: ", sememe_lexion_size)

sememe lexion size:  4975


In [52]:
doc_word_freq = {}
for doc_id,(_, doc_words) in enumerate(reconstruct_def_raw.items()):
    for word in doc_words:
        word_id = sememe_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1
            
word_doc_list = {}
for i,(_, doc_words) in enumerate(reconstruct_def_raw.items()):
    appeared = set()
    for word in doc_words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)
        
word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)
    
tfidf_word2doc_all = {}
for i,(title, doc_words) in enumerate(reconstruct_def_raw.items()):
    doc_word_set = set()
    for word in doc_words:
        if word in doc_word_set or word == title:
            continue
        j = sememe_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        idf = log(1.0 * len(reconstruct_def_raw) / word_doc_freq[sememe_lexion[j]])
        tfidf_word2doc_all[key] = freq * idf
        doc_word_set.add(word)

In [54]:
title_list = [word for word in reconstruct_def_raw.keys()]
tfidf_filter = {}
for title, tfidf in tfidf_word2doc_all.items():
    if tfidf > 3:
        w, d = title.split(",")
        if title_list[int(w)] not in tfidf_filter.keys():
            tfidf_filter[title_list[int(w)]] = []
        tfidf_filter[title_list[int(w)]].append(sememe_lexion[int(d)])
print("all lexion size:", len(tfidf_filter))

all lexion size: 230824


In [55]:
# this part for add pmi weight to building graph
#title_list = [word for word in reconstruct_def_raw.keys()]
#tfidf_filter = {}
#for title, tfidf in tfidf_word2doc_all.items():
#    if tfidf > 3:
#        w, d = title.split(",")
#        if title_list[int(w)] not in tfidf_filter.keys():
#            tfidf_filter[title_list[int(w)]] = []
#        pmi_weight = sememe_lexion[int(d)]+":"+str(tfidf)
#        tfidf_filter[title_list[int(w)]].append(pmi_weight)
#print("all lexion size:", len(tfidf_filter))

In [57]:
tfidf_filter = {k:v for k, v in tfidf_filter.items() if k in reconstruct_def_raw.keys()}

In [58]:
print("all lexion size:", len(tfidf_filter))

all lexion size: 230824


In [64]:
sum([len(v) for v in tfidf_filter.values()])/len(tfidf_filter)

13.592174124007903

In [60]:
title_id_map = {w:i for i,w in enumerate(title_list)}
max_value = 10
sememe_network_wiki = {}
for word, sememes in tfidf_filter.items():
    if len(sememes) != 0:
        if len(sememes) > max_value:
            sememe_freq_max_value = {}
            for sememe in sememes:
                tfidf_word2doc_all_key = str(title_id_map[word]) +","+ str(sememe_id_map[sememe])
                sememe_freq_max_value[sememe] = tfidf_word2doc_all[tfidf_word2doc_all_key]
            sememe_lower = sorted(sememe_freq_max_value.items(),key=lambda item:item[1],reverse=True)[:max_value]
            sememe_lower = [sememe_set[0] for sememe_set in sememe_lower]
            sememe_network_wiki[word] = sememe_lower
        else:
            sememe_network_wiki[word] = sememes
    else:
        continue

In [61]:
sememe_network_wiki = {k:v for k, v in sememe_network_wiki.items() if len(v) != 0}

In [62]:
print("all lexion size:", len(sememe_network_wiki))

all lexion size: 230824


In [63]:
sum([len(v) for v in sememe_network_wiki.values()])/len(sememe_network_wiki)

7.0001776245104494

In [52]:
#sememe_network_pmi = {}
#for k,v in tfidf_filter.items():
#    for vi in v:
#        vi,pmi=vi.split(":")
#        if vi in sememe_network_wiki[k]:
#            if k not in sememe_network_pmi.keys():
#                sememe_network_pmi[k] = []
#            sememe_network_pmi[k].append(vi+":"+pmi)

In [66]:
word_freq = {}
word_set = set()
for words in tfidf_filter.values():
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

print("vocab_size: ", vocab_size)

vocab_size:  4955


In [67]:
sum([len(v) for v in sememe_network_dict_en_wiki_5000.values()])/len(sememe_network_dict_en_wiki_5000)

3.733703057720608

In [68]:
sum([len(v) for v in sememe_network_wiki.values()])/len(sememe_network_wiki)

7.0001776245104494

In [103]:
sememe_network_wiki_ad_dict = dict(sememe_network_dict_en_wiki_5000, **sememe_network_wiki)

In [104]:
len(sememe_network_wiki_ad_dict)

817928

In [105]:
sememe_network_wiki_ad_dict = {k.split(">>>")[0]:[(k.split(">>>")[1],set(v))] for k,v in sememe_network_wiki_ad_dict.items()}

In [106]:
sememe_network_dict_en_wordnet_5000 = np.load("sememe_network_dict_en_wordnet_5000.npy", allow_pickle=True).tolist()

In [107]:
for wn_k,wn_v in sememe_network_dict_en_wordnet_5000.items():
    word = wn_k.split(".")[0]
    if word not in sememe_network_wiki_ad_dict.keys():
        sememe_network_wiki_ad_dict[word] = []
    sememe_network_wiki_ad_dict[word].append((wn_k.split(".")[1],wn_v))

In [108]:
len(sememe_network_wiki_ad_dict)

910369

In [109]:
np.save("skb_ad_dict",sememe_network_wiki_ad_dict)

In [27]:
wordnet_skb = np.load("./data/wordnet_skb.npy", allow_pickle=True).tolist()
wiki_skb = np.load("./data/wiki_skb.npy", allow_pickle=True).tolist()

In [30]:
def _networkskbForm(networkskb):
    '''uniform for NetWorkSKB'''
    networkskb_form = {}
    for wn_k,wn_v in networkskb.items():
        word = wn_k.split(".")[0].replace("_"," ").strip()
        if word not in networkskb_form.keys():
            networkskb_form[word] = []
        networkskb_form[word].append((wn_k.split(".")[1],wn_v))
    return networkskb_form

In [31]:
wordnet_skb = _networkskbForm(wordnet_skb)

In [38]:
for titles, sememe_list in tqdm.tqdm(wiki_skb.items()):
    if " (" in title:
        title = titles.split(" (")[0].strip()
        sense = titles.split(" (")[1].strip()
        sense = sense.replace(")", "").strip() 
    else:
        sense = "n"
    if title not in wordnet_skb.keys():
        wordnet_skb[title] = []
    wordnet_skb[title].append((sense,set(sememe_list)))


100%|█████████████████████████████| 6114239/6114239 [00:13<00:00, 447786.90it/s]


In [40]:
len(wordnet_skb)

152661

In [41]:
np.save("./data/skb_da",wordnet_skb)